In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from snowflake.snowpark import Session

In [2]:
load_dotenv()

True

In [3]:
ACCOUNT = os.getenv('ACCOUNT')
USER = os.getenv('SP_USER')
PASSWORD = os.getenv('PASSWORD')
ROLE = os.getenv('ROLE')
WAREHOUSE = os.getenv('WAREHOUSE')
DATABASE = os.getenv('DATABASE')
SCHEMA = os.getenv('SCHEMA')

print(ACCOUNT, USER, ROLE, WAREHOUSE, DATABASE, SCHEMA)

hum.us-east-2.aws ericatuva RUP UVARND_WH CORE CLIENT


In [4]:
connection_parameters = {
    'account': os.getenv('ACCOUNT'),
    'user': USER,
    'password': os.getenv('PASSWORD'),
    'role': os.getenv('ROLE'),
    'warehouse': os.getenv('WAREHOUSE'),
    'database': os.getenv('DATABASE'),
    'schema': os.getenv('SCHEMA')
}

In [5]:
test_session = Session.builder.configs(connection_parameters).create()

In [6]:
def query_snowpark(query, test_session=test_session):
    print('querying...')
    query_results = test_session.sql(query).collect()
    print('query done')
    
    query_json = list(map(lambda x: x.as_dict(), query_results))
    query_df = pd.DataFrame(query_json)
    
    return query_df

In [7]:
# EVENT META

event_meta_query = """
SELECT
    e.id
    , em.name AS meta_name
    , em.value AS meta_value
FROM event e
LEFT JOIN event_meta em ON e.id = em.event_id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
"""
event_meta_df = query_snowpark(event_meta_query)

querying...
query done


In [8]:
event_meta_df.META_NAME.unique()

array([None, 'referer', 'tags', 'day', 'description', 'title', 'image',
       'content_type', 'utm_campaign', 'utm_content', 'utm_medium',
       'utm_term', 'utm_source'], dtype=object)

In [9]:
unique_events = len(event_meta_df.ID.unique())

In [10]:
unique_events

10863469

### Content Type

In [11]:
event_meta_df[event_meta_df['META_NAME'] == 'content_type']\
    .groupby('META_VALUE')['ID'].nunique()

META_VALUE
journal_article    1068966
microsite_home           4
Name: ID, dtype: int64

- Total Events: 10,863,469
- Journal Articles: 1,068,966
- Microsite Home: 4

### Other Event Meta Types

In [12]:
event_meta_df.groupby('META_NAME')['ID'].nunique()

META_NAME
content_type    1068970
day             2011605
description     1886980
image            279918
referer         2011605
tags             642979
title           1886909
utm_campaign      10174
utm_content       10174
utm_medium        10174
utm_source        10174
utm_term          10174
Name: ID, dtype: int64

In [13]:
content_query = """
SELECT 
    c.type
    , COUNT(DISTINCT e.id) AS events
FROM event e
LEFT JOIN content c ON CONCAT(e.source, '_', e.content_id) = c.id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
GROUP BY 1
"""
content_df = query_snowpark(content_query)

querying...
query done


In [24]:
content_df

,TYPE,EVENTS
0,issue,125641
1,journal_article,8012589
2,account_management,179789
3,search,197223
4,None,1623851
5,in-brief,39
6,self-serve,129483
7,cross-ref-citation,12
8,microsite_home,594842


### Referer & URL

In [14]:
referer_query = """
SELECT
    CASE
        WHEN referer IS NULL THEN 'NULL'
        WHEN referer LIKE '%rupress.org%' THEN 'RUPRESS'
        WHEN referer LIKE '%scholar.google%' THEN 'GOOGLE SCHOLAR'
        WHEN referer LIKE '%google%' THEN 'GOOGLE'
        WHEN referer LIKE '%pubmed%' THEN 'PUBMED'
        ELSE 'OTHER'
    END AS referer_group
    , COUNT(DISTINCT id) AS events
FROM event
WHERE DATE_TRUNC('year', day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

referer_df = query_snowpark(referer_query)

querying...
query done


In [15]:
referer_df

,REFERER_GROUP,EVENTS
0,OTHER,4166569
1,GOOGLE,2933301
2,PUBMED,1428813
3,RUPRESS,1355904
4,GOOGLE SCHOLAR,978882


In [16]:
url_query = """
SELECT
    CASE
        WHEN url LIKE '%article%' THEN 'ARTICLE'
        ELSE 'OTHER'
    END AS url_type
    , COUNT(DISTINCT id) AS events
FROM event
WHERE DATE_TRUNC('year', day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

url_df = query_snowpark(url_query)

querying...
query done


In [17]:
url_df

,URL_TYPE,EVENTS
0,ARTICLE,9516546
1,OTHER,1346923


### Tags & Keywords

In [18]:
tag_query = """
WITH tag_table AS (
    SELECT 
        e.id
        , value AS tag
    FROM event e,
    LATERAL FLATTEN(input => e.tags) f
    WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
)

SELECT
    tag
    , COUNT(DISTINCT id) AS events
FROM tag_table
GROUP BY 1
ORDER BY 2 DESC
"""

tag_df = query_snowpark(tag_query)

querying...
query done


In [19]:
tag_df

,TAG,EVENTS
0,"""mice""",2216857
1,"""t-lymphocytes""",996746
2,"""tissue membrane""",698284
3,"""signal transduction""",619142
4,"""antibodies""",585891
...,...,...
34604,"""N-formylated peptides""",1
34605,"""ciguatoxins""",1
34606,"""osteoclast; islet amyloid polypeptide; CTR; C...",1
34607,"""forehead hematoma""",1


In [20]:
tag_df.head(20)

,TAG,EVENTS
0,"""mice""",2216857
1,"""t-lymphocytes""",996746
2,"""tissue membrane""",698284
3,"""signal transduction""",619142
4,"""antibodies""",585891
5,"""neoplasms""",581571
6,"""infections""",499700
7,"""genes""",425608
8,"""hum_immunopathogenesis""",414480
9,"""mitochondria""",365549


In [21]:
keyword_query = """
SELECT
    ck.keyword
    , COUNT(DISTINCT e.id) AS EVENTS
FROM event e
LEFT JOIN content_keyword ck ON CONCAT(e.source, '_', e.content_id) = ck.content_id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

keyword_df = query_snowpark(keyword_query)

querying...
query done


In [22]:
keyword_df

,KEYWORD,EVENTS
0,mice,2500110
1,None,2125695
2,t-lymphocytes,1133698
3,tissue membrane,768383
4,signal transduction,703489
...,...,...
23662,cochlear implants,1
23663,phosphorylases,1
23664,confidence interval,1
23665,supraoptic nucleus,1


Are tag or keyword groups available in the data?

In [7]:
new_features_query = """
-- NEW CONTENT SCORE
with content_aggs AS (
    SELECT
        COUNT(DISTINCT id) AS total_content
    FROM content

)
,new_content AS (
    SELECT
        *
        , (
            (COALESCE(pdf_click, 0) * 4 * (1 - 0.25))
            + (COALESCE(pageview, 0) * 1 * (1 - 0.25))
            + (COALESCE(post_read_start, 0) * 2 * (1 - 0.2))
            + (COALESCE(post_read_mid, 0) * 5 * (1 - 0.1))
            + (COALESCE(post_read_end, 0) * 6 * (1 - 0.05))
            + (COALESCE(scroll, 0) * 2 * (1 - 0.15))
        ) * (100 / POW(total_content, 0.8)) AS new_content_score
    FROM content
    JOIN content_aggs ON 1 = 1
)
-- USER LEVEL
, event_expanded AS (
    SELECT 
        e.id AS event_id
        , e.set_profile AS profile_id
        , LOWER(e.referer) AS referer
        , e.created
        -- ranking of event per visitor based on created time
        , ROW_NUMBER() OVER (PARTITION BY e.set_profile ORDER BY e.created) AS event_rank
        , p.user_id AS email
        , LOWER(c.type) AS content_type
        , c.id AS content_id
        , c.new_content_score AS content_score
    FROM event e
    LEFT JOIN new_content c ON CONCAT(e.source, '_', e.content_id) = c.id
    LEFT JOIN profile p ON e.set_profile = p.id
    WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
)
, events_with_diff AS (
    SELECT 
        *
        -- subtract created of next event in the list per each visitor
        , DATEDIFF('second', created, LEAD(created) OVER(PARTITION BY profile_id ORDER BY event_rank)) AS seconds_to_next_event
    FROM event_expanded
)
, idle_hours AS (
    SELECT 
        AVG(seconds_to_next_event) AS mean_time
        , 1/AVG(seconds_to_next_event) AS lam
        , (-1 * AVG(seconds_to_next_event) * LN(0.05)) / (60 * 60) AS avg_idle_hours
    FROM events_with_diff
)
, event_aggs AS (
    SELECT
        profile_id
        , COUNT(DISTINCT event_id) AS events
    FROM events_with_diff
    GROUP BY 1
)
, event_aggs_first_40 AS (
    SELECT
        e.profile_id
        , COUNT(DISTINCT CASE WHEN (e.seconds_to_next_event / (60*60)) >= i.avg_idle_hours THEN event_id END) + 1 AS cycles
        , COUNT(DISTINCT CASE WHEN content_type LIKE '%article%' THEN event_id END) AS article_events
        , COUNT(DISTINCT content_id) AS distinct_content
        , COUNT(DISTINCT CASE WHEN content_type LIKE '%article%' THEN content_id END) AS distinct_articles
        , COUNT(DISTINCT CASE WHEN content_type LIKE '%article%' AND referer like '%www.google.com%' THEN content_id END) AS distinct_articles_from_google
        , AVG(content_score) AS average_content_score
    FROM events_with_diff e
    JOIN idle_hours i ON 1 = 1
    WHERE e.event_rank <= 40 -- first 40 events only
    GROUP BY 1
)
, time_to_forty AS (
    SELECT
        a.profile_id
        , DATEDIFF('day', a.created, b.created) AS days_to_40
    FROM event_expanded a
    LEFT JOIN event_expanded b ON a.profile_id = b.profile_id AND b.event_rank = 40
    WHERE a.event_rank = 1
)

SELECT
    ea.profile_id
    , CASE
        WHEN e.events > 40 THEN 1
        ELSE 0
    END AS more_than_40_events
    , ea.cycles AS event_cycles_f40
    , ea.distinct_articles AS distinct_articles_f40
    , CASE
        WHEN ea.distinct_articles > 0 THEN ea.distinct_articles_from_google / ea.distinct_articles
        ELSE 0
    END AS percent_google_articles_f40
    , CASE
        WHEN ea.distinct_content > 0 THEN ea.distinct_articles / ea.distinct_content
        ELSE 0
    END AS percent_article_content_f40
    , COALESCE(ea.average_content_score, 0) AS average_content_score_f40
    , ttf.days_to_40 AS days_to_forty_events
FROM event_aggs_first_40 ea
JOIN event_aggs e ON ea.profile_id = e.profile_id
LEFT JOIN time_to_forty ttf ON ea.profile_id = ttf.profile_id
"""

new_features_df = query_snowpark(new_features_query)

querying...
query done


In [8]:
new_features_df

,PROFILE_ID,MORE_THAN_40_EVENTS,EVENT_CYCLES_F40,DISTINCT_ARTICLES_F40,PERCENT_GOOGLE_ARTICLES_F40,PERCENT_ARTICLE_CONTENT_F40,AVERAGE_CONTENT_SCORE_F40,DAYS_TO_FORTY_EVENTS
0,A_ZSgoIBGoM3uBpF2kSF,1,6,8,0.250000,1.000000,0.000000,70.0
1,kjKCZ4IB8Tq1gs32qWCB,1,4,1,0.000000,0.500000,0.000000,30.0
2,Ji-sNYQBwWEOklUHtLOe,1,2,4,0.000000,0.666667,39.837483,7.0
3,vvCZE4QBwWEOklUHM3Ep,1,4,2,0.500000,1.000000,0.000000,42.0
4,VeHWX4IB8Tq1gs32ty7z,1,8,2,0.000000,1.000000,0.000000,112.0
...,...,...,...,...,...,...,...,...
1889034,qrkuY4MB28LFeUkXyu92,0,1,1,0.000000,1.000000,0.000000,NaN
1889035,qWNj3YMBu9aDgDX1IgAj,0,1,1,1.000000,1.000000,0.000000,NaN
1889036,d4J0iIAB-TISWGBhpe51,0,2,2,0.500000,1.000000,0.000000,NaN
1889037,tJyfX4MB28LFeUkXb816,0,2,1,0.000000,0.500000,119.973921,NaN


In [9]:
# > 40 events
new_features_df[new_features_df['MORE_THAN_40_EVENTS'] == 1]

,PROFILE_ID,MORE_THAN_40_EVENTS,EVENT_CYCLES_F40,DISTINCT_ARTICLES_F40,PERCENT_GOOGLE_ARTICLES_F40,PERCENT_ARTICLE_CONTENT_F40,AVERAGE_CONTENT_SCORE_F40,DAYS_TO_FORTY_EVENTS
0,A_ZSgoIBGoM3uBpF2kSF,1,6,8,0.250000,1.000000,0.000000,70.0
1,kjKCZ4IB8Tq1gs32qWCB,1,4,1,0.000000,0.500000,0.000000,30.0
2,Ji-sNYQBwWEOklUHtLOe,1,2,4,0.000000,0.666667,39.837483,7.0
3,vvCZE4QBwWEOklUHM3Ep,1,4,2,0.500000,1.000000,0.000000,42.0
4,VeHWX4IB8Tq1gs32ty7z,1,8,2,0.000000,1.000000,0.000000,112.0
...,...,...,...,...,...,...,...,...
1830325,lCDhh4IBGoM3uBpFvamA,1,1,3,0.666667,0.230769,41.845592,1.0
1830328,ajcJaIIB8Tq1gs32PCYb,1,8,4,0.250000,1.000000,0.000000,123.0
1830330,-AQuoIIBGoM3uBpF_07P,1,1,1,0.000000,1.000000,0.000000,3.0
1830331,ITAUCoIBUgM-564P3GZN,1,1,7,0.000000,1.000000,0.000000,1.0


In [10]:
# <= 40 events
new_features_df[new_features_df['MORE_THAN_40_EVENTS'] == 0]

,PROFILE_ID,MORE_THAN_40_EVENTS,EVENT_CYCLES_F40,DISTINCT_ARTICLES_F40,PERCENT_GOOGLE_ARTICLES_F40,PERCENT_ARTICLE_CONTENT_F40,AVERAGE_CONTENT_SCORE_F40,DAYS_TO_FORTY_EVENTS
13,m5WJTIQBEEdskq5q7TNY,0,1,8,0.000000,0.800000,51.117406,0.0
61,joRgB4MB28LFeUkX0l2V,0,2,8,0.750000,1.000000,0.000000,19.0
107,3VUJ0YMBu9aDgDX1bdUl,0,5,3,0.000000,0.750000,0.000000,53.0
108,LNOz6YIB28LFeUkXH7g_,0,6,5,0.000000,1.000000,0.000000,119.0
123,xkpgj4IBGoM3uBpF2J1S,0,5,10,1.000000,1.000000,0.000000,126.0
...,...,...,...,...,...,...,...,...
1889034,qrkuY4MB28LFeUkXyu92,0,1,1,0.000000,1.000000,0.000000,NaN
1889035,qWNj3YMBu9aDgDX1IgAj,0,1,1,1.000000,1.000000,0.000000,NaN
1889036,d4J0iIAB-TISWGBhpe51,0,2,2,0.500000,1.000000,0.000000,NaN
1889037,tJyfX4MB28LFeUkXb816,0,2,1,0.000000,0.500000,119.973921,NaN


In [11]:
new_features_df.to_csv('new_features.csv', index = 0)

In [12]:
test_session.close()
print('session closed')

session closed
